In [20]:
# !pip3 install plotly --quiet
# !pip3 install nbformat --quiet

In [21]:
import duckdb 
import pandas as pd
import plotly.express as px
import plotly
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

In [22]:
con = duckdb.connect()
log_db_name = 'benchmark_log_python_3_repeat_no_OOM_test.db'
attach_results = con.sql(f"ATTACH '{log_db_name}' as sqlite_db (TYPE SQLITE)")
ctas_results = con.sql("""
                       create or replace table duckdb_results as 
                       select * from sqlite_db.results
                       where 
                            NOT (benchmark = '003 Convert to Enums' and try_cast(time as float) < 0.01)""")
other_log_db_name = 'benchmark_log_python_no_scale_0103_100_latest.db'
try:
    detach_results = con.sql(f"""DETACH "{other_log_db_name}" """)    
except:
    print(f"Unable to detach {other_log_db_name} - it may not be attached yet")
attach_results = con.sql(f"""ATTACH '{other_log_db_name}' as "{other_log_db_name}" (TYPE SQLITE)""")
insert_results = con.sql(f"""
    insert into duckdb_results 
        select * from "{other_log_db_name}".results
    """)


results = con.sql("""
    with version_ranks as (
        from duckdb_results
        select distinct
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as version,
            version.string_split('.') as split_version,
            try_cast(split_version[1] as int) * 100 * 100 as first_digit,
            try_cast(split_version[2] as int) * 100 as second_digit,
            try_cast((split_version[3]).string_split('-')[1] as int) as third_digit,
            first_digit + second_digit + third_digit as version_rank,
    ), max_version as (
        from version_ranks
        select 
            max(version_rank),
            arg_max(version, version_rank) as max_version
    ), median_results as (
        from duckdb_results
        select 
            run_id,
            min(repeat_id) as repeat_id,
            benchmark,
            scenario,
            median(time) as time,
        group by all            
    )
    from median_results
    join max_version on 1=1
    select 
        * exclude time,
        time as "Time (seconds)",
        (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as "DuckDB Version",
        case when benchmark ilike '%queries%' or benchmark ilike '%enums%' then 'Analysis'
             when benchmark ilike '%csv%' then 'CSV Import' 
             when benchmark ilike '%scan%' or benchmark ilike '%query pandas%' then 'Scan other formats' 
             when benchmark ilike '%export%' then 'Export'
        end as benchmark_type,
        case when benchmark_type = 'CSV Import' then 1
             when benchmark_type = 'Scan other formats' then 2
             when benchmark_type = 'Analysis' then 3
             when benchmark_type = 'Export' then 4
             end as benchmark_type_order,
        benchmark_type as "Benchmark Type",
        sum(time) over (partition by "DuckDB Version") as total_time_by_version,
        sum(case when (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') = max_version.max_version then time else 0 end) over () as total_time_max_version,
        time / total_time_max_version as "Time Relative to Latest Version"
    order by benchmark, run_id, scenario, repeat_id""").df()
results.head(10)

Unable to detach benchmark_log_python_no_scale_0103_100_latest.db - it may not be attached yet


,run_id,repeat_id,benchmark,scenario,max(version_rank),max_version,Time (seconds),DuckDB Version,benchmark_type,benchmark_type_order,Benchmark Type,total_time_by_version,total_time_max_version,Time Relative to Latest Version
0,1,0,001 Query pandas,"{""duckdb_version"": ""v0.10.2""}",10001,1.0.1-dev1125,0.002314,0.10.2,Scan other formats,2.0,Scan other formats,36.549050,42.046638,0.000055
1,1,0,001 Query pandas,"{""duckdb_version"": ""v0.10.3""}",10001,1.0.1-dev1125,0.002345,0.10.3,Scan other formats,2.0,Scan other formats,41.969879,42.046638,0.000056
2,2,0,001 Query pandas,"{""duckdb_version"": ""v0.10.1""}",10001,1.0.1-dev1125,0.001823,0.10.1,Scan other formats,2.0,Scan other formats,37.606047,42.046638,0.000043
3,2,0,001 Query pandas,"{""duckdb_version"": ""v1.0.0""}",10001,1.0.1-dev1125,0.002033,1.0.0,Scan other formats,2.0,Scan other formats,42.411518,42.046638,0.000048
4,3,0,001 Query pandas,"{""duckdb_version"": ""v0.10.0""}",10001,1.0.1-dev1125,0.002377,0.10.0,Scan other formats,2.0,Scan other formats,38.160447,42.046638,0.000057
5,3,0,001 Query pandas,"{""duckdb_version"": ""v1.0.1-dev1125""}",10001,1.0.1-dev1125,0.002486,1.0.1-dev1125,Scan other formats,2.0,Scan other formats,42.046638,42.046638,0.000059
6,4,0,001 Query pandas,"{""duckdb_version"": ""v0.9.2""}",10001,1.0.1-dev1125,0.001576,0.9.2,Scan other formats,2.0,Scan other formats,40.665796,42.046638,0.000037
7,5,0,001 Query pandas,"{""duckdb_version"": ""v0.9.1""}",10001,1.0.1-dev1125,0.001964,0.9.1,Scan other formats,2.0,Scan other formats,41.170681,42.046638,0.000047
8,6,0,001 Query pandas,"{""duckdb_version"": ""0.9.0""}",10001,1.0.1-dev1125,0.002030,0.9.0,Scan other formats,2.0,Scan other formats,41.512780,42.046638,0.000048
9,7,0,001 Query pandas,"{""duckdb_version"": ""0.8.1""}",10001,1.0.1-dev1125,0.002295,0.8.1,Scan other formats,2.0,Scan other formats,111.916529,42.046638,0.000055


In [23]:
con.sql("""select sum(time) / 60.0 from duckdb_results""").df()

,"(sum(""time"") / 60.0)"
0,188.208973


In [24]:
test_results = con.sql("""
    with version_ranks as (
        from duckdb_results
        select distinct
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as version,
            version.string_split('.') as split_version,
            try_cast(split_version[1] as int) * 100 * 100 as first_digit,
            try_cast(split_version[2] as int) * 100 as second_digit,
            try_cast((split_version[3]).string_split('-')[1] as int) as third_digit,
            first_digit + second_digit + third_digit as version_rank,
    )
    from version_ranks
    select 
        max(version_rank),
        arg_max(version, version_rank) as max_version
""").df()
test_results

,max(version_rank),max_version
0,10001,1.0.1-dev1125


In [25]:
fig = px.area(
    results,
    x="DuckDB Version",
    y="Time (seconds)",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis","Export"]},
    template='plotly_white'
)
print(plotly.io.to_json(fig, pretty=True))
fig.show()

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "Benchmark Type=CSV Import\u003cbr\u003ebenchmark=002 Create table from csv\u003cbr\u003eDuckDB Version=%{x}\u003cbr\u003eTime (seconds)=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "CSV Import",
      "line": {
        "color": "#636efa"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "CSV Import",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "0.10.2",
        "0.10.3",
        "0.10.1",
        "1.0.0",
        "0.10.0",
        "1.0.1-dev1125",
        "0.9.2",
        "0.9.1",
        "0.9.0",
        "0.8.1",
        "0.7.1",
        "0.6.1",
        "0.5.1",
        "0.4.0",
        "0.3.4",
        "0.3.2",
        "0.3.1",
        "0.3.0",
        "0.2.9",
        "0.2.8",
        "0.2.7"
      ],
      "xaxis": "x",
      "y": [
        1.4280079589999986,
 

In [26]:
fig = px.area(
    results,
    x="DuckDB Version",
    y="Time Relative to Latest Version",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis","Export"]},
    template='plotly_white'
)
fig.show()

In [27]:
# Pandas or Arrow
# Pick either Arrow or Pandas for import/export depending on which is faster for each version
pandas_or_arrow = con.sql("""
    with version_ranks as (
        from duckdb_results
        select distinct
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as version,
            version.string_split('.') as split_version,
            try_cast(split_version[1] as int) * 100 * 100 as first_digit,
            try_cast(split_version[2] as int) * 100 as second_digit,
            try_cast((split_version[3]).string_split('-')[1] as int) as third_digit,
            first_digit + second_digit + third_digit as version_rank,
    ), max_version as (
        from version_ranks
        select 
            max(version_rank),
            arg_max(version, version_rank) as max_version
    ), median_results as (
        from duckdb_results
        select 
            run_id,
            min(repeat_id) as repeat_id,
            benchmark,
            scenario,
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as version,
            median(time) as time,
        where 
            benchmark != '001 Query pandas'
        group by all            
    )
    from median_results
    join max_version on 1=1
    join version_ranks on median_results.version = version_ranks.version 
    select 
        median_results.* exclude (time, version, benchmark),
        benchmark.replace('Windowing performance test: ','') as benchmark,
        version_ranks.version_rank,
        time as "Time (seconds)",
        version_ranks.version as "DuckDB Version",
        case when (benchmark ilike '%queries%' or benchmark ilike '%enums%') and benchmark ilike '%join%' then 'Analysis - Join'
             when (benchmark ilike '%queries%' or benchmark ilike '%enums%') and benchmark not ilike '%join%' then 'Analysis - Group By'
             when benchmark ilike '%csv%' then 'CSV Import' 
             when benchmark ilike '%scan%' or benchmark ilike '%query pandas%' then 'Scan other formats' 
             when benchmark ilike '%export%' then 'Export'
             when benchmark ilike '%window%' then 'Window Functions' 
        end as benchmark_type,
        case when benchmark_type = 'CSV Import' then 1
             when benchmark_type = 'Scan other formats' then 2
             when benchmark_type = 'Analysis - Group By' then 3
             when benchmark_type = 'Analysis - Join' then 4
             when benchmark_type = 'Window Functions' then 5
             when benchmark_type = 'Export' then 6
             end as benchmark_type_order,
        benchmark_type as "Benchmark Type",
        sum(time) over (partition by "DuckDB Version") as total_time_by_version,
        sum(case when (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') = max_version.max_version then time else 0 end) over () as total_time_max_version,
        time / total_time_max_version as "Time Relative to Latest Version",
        sum(case when (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') = max_version.max_version then time else 0 end) over (partition by benchmark_type) as benchmark_type_time_max_version,
        time / benchmark_type_time_max_version as "Time Relative to Latest Version by Type",
        sum(case when (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') = max_version.max_version then time else 0 end) over (partition by benchmark_type, benchmark) as benchmark_time_max_version,
        time / benchmark_time_max_version as "Time Relative to Latest Version by Benchmark",
        case when benchmark ilike '%pandas%' then 'Pandas' 
             when benchmark ilike '%arrow%' then 'Arrow'
             end as pandas_or_arrow
    where 
        (version_rank < 501 and (pandas_or_arrow is null OR pandas_or_arrow = 'Pandas'))
        OR (version_rank >= 501 and (pandas_or_arrow is null OR pandas_or_arrow = 'Arrow'))
    order by benchmark, run_id, version_rank, repeat_id""").df()
pandas_or_arrow.head()

,run_id,repeat_id,scenario,benchmark,version_rank,Time (seconds),DuckDB Version,benchmark_type,benchmark_type_order,Benchmark Type,total_time_by_version,total_time_max_version,Time Relative to Latest Version,benchmark_type_time_max_version,Time Relative to Latest Version by Type,benchmark_time_max_version,Time Relative to Latest Version by Benchmark,pandas_or_arrow
0,1,0,"{""duckdb_version"": ""v0.10.2""}",002 Create table from csv,1002,1.428008,0.10.2,CSV Import,1,CSV Import,29.487007,34.24914,0.041695,6.446106,0.221530,1.809174,0.789315,None
1,1,0,"{""duckdb_version"": ""v0.10.3""}",002 Create table from csv,1003,1.880705,0.10.3,CSV Import,1,CSV Import,34.675821,34.24914,0.054912,6.446106,0.291758,1.809174,1.039538,None
2,2,0,"{""duckdb_version"": ""v0.10.1""}",002 Create table from csv,1001,1.402783,0.10.1,CSV Import,1,CSV Import,30.099737,34.24914,0.040958,6.446106,0.217617,1.809174,0.775372,None
3,2,0,"{""duckdb_version"": ""v1.0.0""}",002 Create table from csv,10000,1.823720,1.0.0,CSV Import,1,CSV Import,34.908619,34.24914,0.053249,6.446106,0.282918,1.809174,1.008040,None
4,3,0,"{""duckdb_version"": ""v0.10.0""}",002 Create table from csv,1000,1.408106,0.10.0,CSV Import,1,CSV Import,30.631541,34.24914,0.041114,6.446106,0.218443,1.809174,0.778314,None


In [28]:
# We need to fix plotly's ordering issue when we filter down to just analysis
pandas_or_arrow_with_placeholders = con.sql("""
    
    with version_ranks as (
        from duckdb_results
        select distinct
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as version,
            version.string_split('.') as split_version,
            try_cast(split_version[1] as int) * 100 * 100 as first_digit,
            try_cast(split_version[2] as int) * 100 as second_digit,
            try_cast((split_version[3]).string_split('-')[1] as int) as third_digit,
            first_digit + second_digit + third_digit as version_rank,
    ), placeholder as (
        from duckdb_results
        join version_ranks on version_ranks.version = (duckdb_results.scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev')
        join (select distinct "Benchmark Type" from pandas_or_arrow) benchmark_types on 1=1
        select 
            0 as run_id,
            min(repeat_id) as repeat_id,
            '000 Placeholder for charting' as benchmark,
            scenario,
            (scenario::json ->> 'duckdb_version').replace('v','').replace('de', 'dev') as "DuckDB Version",
            0 as "Time (seconds)",
            0 as "Time Relative to Latest Version",   
            version_ranks.version_rank,
            "Benchmark Type" 
        group by all
    ), unioned_with_placeholder as (
        from pandas_or_arrow
        union all by name
        from placeholder   
    )
    from unioned_with_placeholder
    order by benchmark, run_id, version_rank, repeat_id
              
""").df()
# pandas_or_arrow_with_placeholders

In [29]:
fig = px.area(
    pandas_or_arrow_with_placeholders,
    x="DuckDB Version",
    y="Time (seconds)",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.T10,
)
print(plotly.io.to_json(fig, pretty=True))
fig.show()

fig = px.area(
    pandas_or_arrow_with_placeholders,
    x="DuckDB Version",
    y="Time Relative to Latest Version",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.T10,
)
print(plotly.io.to_json(fig, pretty=True))
fig.show()



{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "Benchmark Type=CSV Import\u003cbr\u003ebenchmark=000 Placeholder for charting\u003cbr\u003eDuckDB Version=%{x}\u003cbr\u003eTime (seconds)=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "CSV Import",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "CSV Import",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "0.2.7",
        "0.2.8",
        "0.2.9",
        "0.3.0",
        "0.3.1",
        "0.3.2",
        "0.3.4",
        "0.4.0",
        "0.5.1",
        "0.6.1",
        "0.7.1",
        "0.8.1",
        "0.9.0",
        "0.9.1",
        "0.9.2",
        "0.10.0",
        "0.10.1",
        "0.10.2",
        "0.10.3",
        "1.0.0",
        "1.0.1-dev1125"
      ],
      "xaxis": "x",
      "y": [
        0.0,
        0.0,


{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "Benchmark Type=CSV Import\u003cbr\u003ebenchmark=000 Placeholder for charting\u003cbr\u003eDuckDB Version=%{x}\u003cbr\u003eTime Relative to Latest Version=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "CSV Import",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "CSV Import",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "0.2.7",
        "0.2.8",
        "0.2.9",
        "0.3.0",
        "0.3.1",
        "0.3.2",
        "0.3.4",
        "0.4.0",
        "0.5.1",
        "0.6.1",
        "0.7.1",
        "0.8.1",
        "0.9.0",
        "0.9.1",
        "0.9.2",
        "0.10.0",
        "0.10.1",
        "0.10.2",
        "0.10.3",
        "1.0.0",
        "1.0.1-dev1125"
      ],
      "xaxis": "x",
      "y": [
        0

In [30]:
benchmark_types = ["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]
for benchmark_type in benchmark_types:
    fig = px.line(
        con.sql(f"""from pandas_or_arrow_with_placeholders where "Benchmark Type" = '{benchmark_type}'""").df(),
        x="DuckDB Version",
        y="Time Relative to Latest Version by Benchmark",
        color="benchmark",
        # line_group="benchmark",
        # category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
        template='plotly_white',
        color_discrete_sequence=px.colors.qualitative.T10,
        title=benchmark_type,
    )
    fig.show()

In [31]:
benchmark_types = ["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]
for benchmark_type in benchmark_types:
    fig = px.area(
        con.sql(f"""from pandas_or_arrow_with_placeholders where "Benchmark Type" = '{benchmark_type}'""").df(),
        x="DuckDB Version",
        y="Time Relative to Latest Version by Type",
        color="benchmark",
        # line_group="benchmark",
        # category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
        template='plotly_white',
        color_discrete_sequence=px.colors.qualitative.T10,
        title=benchmark_type,
    )
    fig.show()

- Group by
    - 0.3.1: 1.28 - .48 improvement
        - Default multithreading (https://github.com/duckdb/duckdb/pull/2393)
        - Push based execution (https://github.com/duckdb/duckdb/pull/2393)
    - 0.6.1:  0.32 to 0.1 improvement
        - Parallel data loading (https://github.com/duckdb/duckdb/pull/5082)
        - Patas floating point compression (https://github.com/duckdb/duckdb/pull/5044)
    - 0.8.1: from .1 to .8 improvement
        - Avoid unnecessary resizing of hash tables (https://github.com/duckdb/duckdb/pull/6877)
    - 0.10.0: from .8 to .75
- Join
    - 0.6.1: 2.2 to 0.35
        - Improvements to out of core hash join (https://github.com/duckdb/duckdb/pull/4970)
        - Parallel data loading (https://github.com/duckdb/duckdb/pull/5082)
        - Patas floating point compression (https://github.com/duckdb/duckdb/pull/5044)
    - 0.7.1: 0.35 to 0.2
        - *TODO NEED TO ADD THINGS HERE*
    - (then gave back ground to 0.27 up through 0.9.2)
    - 0.10.0: 0.27 to 0.2
- 008 Scan and Aggregate over Parquet file
    - 0.3.1: .005 to .001 
        - Default multithreading (https://github.com/duckdb/duckdb/pull/2393)
        - Push based execution (https://github.com/duckdb/duckdb/pull/2393)
- Create table from csv
    - 0.8.1: 1.2 to 0.4
        - Parallel CSV Reader as default (https://github.com/duckdb/duckdb/pull/6977)
    - 0.10.0: 0.36 to 0.28
- Scan and aggregate over Pandas
    - 668 - 271
- Should tell the Pandas vs. Arrow story (0.5.1)
- Should tell the Enums story (0.6.1)
- I need to do this for window functions
- Benchmark the connection time?
    - Lazily load row groups from tables: 100x benefit with 770 million rows: https://github.com/duckdb/duckdb/pull/6715

In [32]:
# Also plot by date
from datetime import datetime
versions = {
    # 0.2.7 is the first with MacOS ARM
    '0.2.7': {'date':datetime.strptime('2021-06-14','%Y-%m-%d')},
    '0.2.8': {'date':datetime.strptime('2021-08-02','%Y-%m-%d')},
    '0.2.9': {'date':datetime.strptime('2021-09-06','%Y-%m-%d')},
    '0.3.0': {'date':datetime.strptime('2021-10-06','%Y-%m-%d')},
    '0.3.1': {'date':datetime.strptime('2021-11-16','%Y-%m-%d')},
    '0.3.2': {'date':datetime.strptime('2022-02-07','%Y-%m-%d')},
    # 0.3.3 did not upload to pip correctly so it should be skipped
    # '0.3.3': {'date':datetime.strptime('2022-04-11','%Y-%m-%d')},
    '0.3.4': {'date':datetime.strptime('2022-04-25','%Y-%m-%d')},
    '0.4.0': {'date':datetime.strptime('2022-06-20','%Y-%m-%d')},
    '0.5.1': {'date':datetime.strptime('2022-09-19','%Y-%m-%d')},
    '0.6.1': {'date':datetime.strptime('2022-12-06','%Y-%m-%d')},
    '0.7.1': {'date':datetime.strptime('2023-02-27','%Y-%m-%d')},
    '0.8.1': {'date':datetime.strptime('2023-06-13','%Y-%m-%d')},
    '0.9.0': {'date':datetime.strptime('2023-09-26','%Y-%m-%d')},
    '0.9.1': {'date':datetime.strptime('2023-10-11','%Y-%m-%d')},
    '0.9.2': {'date':datetime.strptime('2023-11-14','%Y-%m-%d')},
    '0.10.0': {'date':datetime.strptime('2024-02-13','%Y-%m-%d')},
    '0.10.1': {'date':datetime.strptime('2024-03-18','%Y-%m-%d')},
    '0.10.2': {'date':datetime.strptime('2024-04-17','%Y-%m-%d')},
    '0.10.3': {'date':datetime.strptime('2024-05-22','%Y-%m-%d')},
    '1.0.0': {'date':datetime.strptime('2024-06-03','%Y-%m-%d')},
    'latest': {'date':datetime.now()},
}
versions_df = pd.DataFrame([versions])
# versions_df
pandas_or_arrow_with_dates = con.sql("""
    with unpivoted as (
        unpivot versions_df
        on columns(*)
        into name version
             value date_struct
    ), version_to_date_map as (
        from unpivoted 
        select 
            version,
            date_struct.date as "Release Date"
    )
    from pandas_or_arrow_with_placeholders data
    left join version_to_date_map on data."DuckDB Version" = version_to_date_map.version
    select 
        data.*,
        version_to_date_map.* exclude version
        
""").df()
# pandas_or_arrow_with_dates

In [33]:
fig = px.area(
    pandas_or_arrow_with_dates,
    x="Release Date",
    y="Time (seconds)",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.T10,
    hover_data=["DuckDB Version"]
)
print(plotly.io.to_json(fig, pretty=True))
# fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'})
# print(fig.data[1])
# fig.data[1].update(xaxis='x2')
fig.show()

fig = px.area(
    pandas_or_arrow_with_dates,
    x="Release Date",
    y="Time Relative to Latest Version",
    color="Benchmark Type",
    line_group="benchmark",
    category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.T10,
    hover_data=["DuckDB Version"]
)
print(plotly.io.to_json(fig, pretty=True))
fig.show()

{
  "data": [
    {
      "customdata": [
        [
          "0.2.7"
        ],
        [
          "0.2.8"
        ],
        [
          "0.2.9"
        ],
        [
          "0.3.0"
        ],
        [
          "0.3.1"
        ],
        [
          "0.3.2"
        ],
        [
          "0.3.4"
        ],
        [
          "0.4.0"
        ],
        [
          "0.5.1"
        ],
        [
          "0.6.1"
        ],
        [
          "0.7.1"
        ],
        [
          "0.8.1"
        ],
        [
          "0.9.0"
        ],
        [
          "0.9.1"
        ],
        [
          "0.9.2"
        ],
        [
          "0.10.0"
        ],
        [
          "0.10.1"
        ],
        [
          "0.10.2"
        ],
        [
          "0.10.3"
        ],
        [
          "1.0.0"
        ],
        [
          "1.0.1-dev1125"
        ]
      ],
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "Benchmark Type=CSV Import\u003cbr\u003ebenc

{
  "data": [
    {
      "customdata": [
        [
          "0.2.7"
        ],
        [
          "0.2.8"
        ],
        [
          "0.2.9"
        ],
        [
          "0.3.0"
        ],
        [
          "0.3.1"
        ],
        [
          "0.3.2"
        ],
        [
          "0.3.4"
        ],
        [
          "0.4.0"
        ],
        [
          "0.5.1"
        ],
        [
          "0.6.1"
        ],
        [
          "0.7.1"
        ],
        [
          "0.8.1"
        ],
        [
          "0.9.0"
        ],
        [
          "0.9.1"
        ],
        [
          "0.9.2"
        ],
        [
          "0.10.0"
        ],
        [
          "0.10.1"
        ],
        [
          "0.10.2"
        ],
        [
          "0.10.3"
        ],
        [
          "1.0.0"
        ],
        [
          "1.0.1-dev1125"
        ]
      ],
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "Benchmark Type=CSV Import\u003cbr\u003ebenc

In [34]:
benchmark_types = ["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]
for benchmark_type in benchmark_types:
    fig = px.line(
        con.sql(f"""from pandas_or_arrow_with_dates where "Benchmark Type" = '{benchmark_type}'""").df(),
        x="Release Date",
        y="Time Relative to Latest Version by Benchmark",
        color="benchmark",
        # line_group="benchmark",
        # category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
        template='plotly_white',
        color_discrete_sequence=px.colors.qualitative.T10,
        title=benchmark_type,
    )
    fig.show()

In [35]:
benchmark_types = ["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]
for benchmark_type in benchmark_types:
    fig = px.area(
        con.sql(f"""from pandas_or_arrow_with_dates where "Benchmark Type" = '{benchmark_type}'""").df(),
        x="Release Date",
        y="Time Relative to Latest Version by Type",
        color="benchmark",
        # line_group="benchmark",
        # category_orders={"Benchmark Type":["CSV Import","Scan other formats","Analysis - Group By", 'Analysis - Join', 'Window Functions', "Export"]},
        template='plotly_white',
        color_discrete_sequence=px.colors.qualitative.T10,
        title=benchmark_type,
    )
    print(plotly.io.to_json(fig, pretty=True))
    fig.show()

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "benchmark=000 Placeholder for charting\u003cbr\u003eRelease Date=%{x}\u003cbr\u003eTime Relative to Latest Version by Type=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "000 Placeholder for charting",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "000 Placeholder for charting",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "2021-06-14T00:00:00",
        "2021-08-02T00:00:00",
        "2021-09-06T00:00:00",
        "2021-10-06T00:00:00",
        "2021-11-16T00:00:00",
        "2022-02-07T00:00:00",
        "2022-04-25T00:00:00",
        "2022-06-20T00:00:00",
        "2022-09-19T00:00:00",
        "2022-12-06T00:00:00",
        "2023-02-27T00:00:00",
        "2023-06-13T00:00:00",
        "2023-09-26T00:00:00",
        "2023

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "benchmark=000 Placeholder for charting\u003cbr\u003eRelease Date=%{x}\u003cbr\u003eTime Relative to Latest Version by Type=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "000 Placeholder for charting",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "000 Placeholder for charting",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "2021-06-14T00:00:00",
        "2021-08-02T00:00:00",
        "2021-09-06T00:00:00",
        "2021-10-06T00:00:00",
        "2021-11-16T00:00:00",
        "2022-02-07T00:00:00",
        "2022-04-25T00:00:00",
        "2022-06-20T00:00:00",
        "2022-09-19T00:00:00",
        "2022-12-06T00:00:00",
        "2023-02-27T00:00:00",
        "2023-06-13T00:00:00",
        "2023-09-26T00:00:00",
        "2023

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "benchmark=000 Placeholder for charting\u003cbr\u003eRelease Date=%{x}\u003cbr\u003eTime Relative to Latest Version by Type=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "000 Placeholder for charting",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "000 Placeholder for charting",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "2021-06-14T00:00:00",
        "2021-08-02T00:00:00",
        "2021-09-06T00:00:00",
        "2021-10-06T00:00:00",
        "2021-11-16T00:00:00",
        "2022-02-07T00:00:00",
        "2022-04-25T00:00:00",
        "2022-06-20T00:00:00",
        "2022-09-19T00:00:00",
        "2022-12-06T00:00:00",
        "2023-02-27T00:00:00",
        "2023-06-13T00:00:00",
        "2023-09-26T00:00:00",
        "2023

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "benchmark=000 Placeholder for charting\u003cbr\u003eRelease Date=%{x}\u003cbr\u003eTime Relative to Latest Version by Type=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "000 Placeholder for charting",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "000 Placeholder for charting",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "2021-06-14T00:00:00",
        "2021-08-02T00:00:00",
        "2021-09-06T00:00:00",
        "2021-10-06T00:00:00",
        "2021-11-16T00:00:00",
        "2022-02-07T00:00:00",
        "2022-04-25T00:00:00",
        "2022-06-20T00:00:00",
        "2022-09-19T00:00:00",
        "2022-12-06T00:00:00",
        "2023-02-27T00:00:00",
        "2023-06-13T00:00:00",
        "2023-09-26T00:00:00",
        "2023

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "benchmark=000 Placeholder for charting\u003cbr\u003eRelease Date=%{x}\u003cbr\u003eTime Relative to Latest Version by Type=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "000 Placeholder for charting",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "000 Placeholder for charting",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "2021-06-14T00:00:00",
        "2021-08-02T00:00:00",
        "2021-09-06T00:00:00",
        "2021-10-06T00:00:00",
        "2021-11-16T00:00:00",
        "2022-02-07T00:00:00",
        "2022-04-25T00:00:00",
        "2022-06-20T00:00:00",
        "2022-09-19T00:00:00",
        "2022-12-06T00:00:00",
        "2023-02-27T00:00:00",
        "2023-06-13T00:00:00",
        "2023-09-26T00:00:00",
        "2023

{
  "data": [
    {
      "fillpattern": {
        "shape": ""
      },
      "hovertemplate": "benchmark=000 Placeholder for charting\u003cbr\u003eRelease Date=%{x}\u003cbr\u003eTime Relative to Latest Version by Type=%{y}\u003cextra\u003e\u003c\u002fextra\u003e",
      "legendgroup": "000 Placeholder for charting",
      "line": {
        "color": "#4C78A8"
      },
      "marker": {
        "symbol": "circle"
      },
      "mode": "lines",
      "name": "000 Placeholder for charting",
      "orientation": "v",
      "showlegend": true,
      "stackgroup": "1",
      "x": [
        "2021-06-14T00:00:00",
        "2021-08-02T00:00:00",
        "2021-09-06T00:00:00",
        "2021-10-06T00:00:00",
        "2021-11-16T00:00:00",
        "2022-02-07T00:00:00",
        "2022-04-25T00:00:00",
        "2022-06-20T00:00:00",
        "2022-09-19T00:00:00",
        "2022-12-06T00:00:00",
        "2023-02-27T00:00:00",
        "2023-06-13T00:00:00",
        "2023-09-26T00:00:00",
        "2023

In [36]:
# Want to show a table of the various configurations (Pandas vs. Arrow, Enums or not, etc.)
# Use the DuckDB CLI to generate Markdown
import subprocess 
import os
duckdb_location = './duckdb_0_9_0'
sql_location = './benchmark_configs.sql'
versions_with_enums = ['0.6.1', '0.7.1', '0.8.1', '0.9.0', '0.9.1', '0.9.2', '0.10.0', '0.10.1', '0.10.2']
versions_using_pandas  = ['0.2.7', '0.2.8', '0.2.9', '0.3.0', '0.3.1', '0.3.2', '0.3.4', '0.4.0']
versions_using_pyarrow = ['0.5.1', '0.6.1', '0.7.1', '0.8.1', '0.9.0', '0.9.1', '0.9.2', '0.10.0', '0.10.1', '0.10.2']

# versions_without_enums_pre_pandas = []
# for version in versions_without_enums:
#     versions_without_enums_pre_pandas.append({'duckdb_version_without_enums':version})
# versions_without_enums_df = pd.DataFrame(versions_without_enums_pre_pandas)
# print(versions_without_enums_df)

# versions_without_pyarrow_pre_pandas = []
# for version in versions_without_pyarrow:
#     versions_without_pyarrow_pre_pandas.append({'duckdb_version_without_pyarrow':version})
# versions_without_pyarrow_df = pd.DataFrame(versions_without_pyarrow_pre_pandas)
# print(versions_without_pyarrow_df)

if os.path.exists(sql_location):
    os.remove(sql_location)
with open(sql_location, mode='w') as f:
    f.write(f""".mode markdown
            with pandas as (
                SELECT UNNEST({versions_using_pandas}) as versions_using_pandas
            ), enums as (
                SELECT UNNEST({versions_with_enums}) as versions_with_enums
            ), pyarrow as (
                SELECT UNNEST({versions_using_pyarrow}) as versions_using_pyarrow
            ), all_versions as (
                from pandas select distinct versions_using_pandas as duckdb_version
                union
                from enums select distinct versions_with_enums as duckdb_version
                union 
                from pyarrow select distinct *
            )
            from all_versions a
            left join pandas on a.duckdb_version = pandas.versions_using_pandas 
            left join pyarrow on a.duckdb_version = pyarrow.versions_using_pyarrow
            left join enums on a.duckdb_version = enums.versions_with_enums
            select 
                a.*,
                a.duckdb_version.string_split('.') as split_version,
                try_cast(split_version[1] as int) * 100 * 100 as first_digit,
                try_cast(split_version[2] as int) * 100 as second_digit,
                try_cast(split_version[3] as int) as third_digit,
                first_digit + second_digit + third_digit as version_rank,
                pandas.*,
                pyarrow.*,
                enums.*,
            order by 
                version_rank
""")
result = subprocess.run([duckdb_location, ':memory:', "-c",f""".read {sql_location}"""], capture_output=True, text=True)
# print(result)
print('result.stdout:\n'+result.stdout)
print('result.stderr:\n'+result.stderr)

result.stdout:
| duckdb_version | split_version | first_digit | second_digit | third_digit | version_rank | versions_using_pandas | versions_using_pyarrow | versions_with_enums |
|----------------|---------------|-------------|--------------|-------------|--------------|-----------------------|------------------------|---------------------|
| 0.2.7          | [0, 2, 7]     | 0           | 200          | 7           | 207          | 0.2.7                 |                        |                     |
| 0.2.8          | [0, 2, 8]     | 0           | 200          | 8           | 208          | 0.2.8                 |                        |                     |
| 0.2.9          | [0, 2, 9]     | 0           | 200          | 9           | 209          | 0.2.9                 |                        |                     |
| 0.3.0          | [0, 3, 0]     | 0           | 300          | 0           | 300          | 0.3.0                 |                        |                     |
|

## Brainstorming
- Total time area plot
- Relative time area plot (with current as 1, scaling the rest off of that)
    - Grouping related operations together
        - Maybe grouping by type (export to pandas)
        - Maybe grouping by task (group by tests)
        - Group by "importing, querying, exporting" to show the whole burger idea
    - Likely need to filter things down
    - Choosing between Pandas and Arrow whenever Arrow was faster
- Annotating the charts to show what improved with each version

- Max data scale plot
    - Likely need more than just 2 datasets (10 million, 100 million, 1 billion, 10 billion?) Likely unable to generate 10 billion locally with this script.
    - Probably want a column chart with group by and join as the 2 types of bars (alternating group by / join for each version showing the max it can handle? Or maybe a line chart)
